In [2]:
from Preprocessing.data_clean import get_data, dropping_no_betting_data
from sklearn.preprocessing import MinMaxScaler
from sklearn import set_config
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout
from keras.utils import get_custom_objects
import pandas as pd
import numpy as np
import pickle
import warnings

In [3]:
"""1)  Getting the data and first round of preprocessing"""
###receiving all columns of data for user selected rows (by track and time)

# #Get the data from the bucket
data = get_data("../../../raw_data/hr_data_0409_221rem.csv") ### CHANGE THIS PATH to get data from the bucket
print("data loaded")

#Fill in the missing odds
# data = dropping_no_betting_data(data)
data = data.dropna(subset=['f_bsp', 'f_pm_15m', 'f_pm_10m', 'f_pm_05m' , 'f_pm_03m', 'f_pm_02m', 'f_pm_01m'], how='all')
def impute_row(row):
    row = row.fillna(method='ffill').fillna(method='bfill')
    return row
columns_to_impute = ['f_bsp', 'f_pm_15m', 'f_pm_10m', 'f_pm_05m', 'f_pm_03m', 'f_pm_02m', 'f_pm_01m']
data[columns_to_impute] = data[columns_to_impute].apply(impute_row, axis=1)
#Drop stall_position NAs
data = data[data['stall_position'].notna()]
#Remove horses with odds over 50 at 5m before the race
data = data[(data['f_pm_05m'] <= 50)]
#Reset index
data = data.reset_index(drop=True)

/Users/joshstone/code/lucasglanville/and_theyre_off_backend/Preprocessing/data_clean.py:7: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(LOCAL_FILEPATH)


data acquired with shape (120675, 116)
data loaded


In [4]:
data = data.iloc[70000:]

In [6]:
#######################################################


"""2)  Filling the null L16 columns with 0s"""

X_preproc = data[[
        'iv_trainer_l16r', 'iv_jockey_l16r',
        'ae_trainer_l16r' ,'ae_jockey_l16r']]
X_preproc = X_preproc.fillna(0)

#######################################################

"""3)  Scaling the numerical values and defining X"""

#Adding f_runners and stall_position to X_preproc pre-scaling
X_preproc['f_runners'] = data['f_runners']
X_preproc['stall_position'] = data['stall_position']


#Loading scaler values and scaling 5 features
set_config(transform_output = "pandas")
with open('../../../api/scaler_updated2.pkl', 'rb') as f: # CHANGE THIS PATH to get the saved scalar
  loaded_scaler = pickle.load(f)
X = loaded_scaler.transform(X_preproc)

#Adding final 2 features that don't need scaling
X['pred_isp_prob'] = 1 / data['pred_isp']

#Matching the column order to the order of the original saved weights
X = X[['stall_position', 'iv_trainer_l16r', 'iv_jockey_l16r', 'ae_trainer_l16r', 'ae_jockey_l16r', 'pred_isp_prob', 'f_runners']]

#######################################################

#"""4)  Defining y"""

# NOT NEEDED IN CURRENT DESIGN (but keeping incase it becomes useful)

# def winner(x):
#         if x == 1:
#             return 1
#         else:
#             return 0
# data['f_place'] = data['f_place'].apply(winner)
# y = data[['f_place', 'pred_isp']]

# y = data[['f_place', 'pred_isp']]

#######################################################

"""5) Defining backtest and changing commision to 2%"""

# Define a function to create a new profit column with 2% commision
def fill_01m_profit(data):
    def fill_nan(row):
        if row['f_place'] == 0:
            return -1
        elif row['f_place'] == 1:
            return (row['f_pm_01m'] - 1) * 0.98
        else:
            return row['f_pm_01m_p_back']

# Apply the lambda function to create 01m_profit column
    data['01m_profit'] = data.apply(fill_nan, axis=1)
    return data

data = fill_01m_profit(data)

backtest = data[['f_ko', 'f_track', 'f_id', 'id','f_horse','f_pm_01m', '01m_profit', 'f_place']]

#######################################################

"""6)  Model Architecture"""

NN = Sequential()
NN.add(InputLayer(input_shape=(7, ))) # input layer
NN.add(Dense(32, activation='relu')) # hidden layer 1
NN.add(Dense(2, activation='softmax')) # output layer

#######################################################

"""7)  Loading Weights"""


NN.load_weights("../../../api/custom_scorer0.05_7input_l16_05mfilter_01mplace") ##CHANGE PATH TO LOAD MODEL WEIGHTS


#######################################################

"""8)  Creating preds"""

y_pred = NN.predict(X)

#######################################################

"""9) Creating backtest table"""

backtest['model_preds'] = y_pred[:, 0:1]
backtest['model_preds'] = round(backtest['model_preds'],2)
backtest = backtest.sort_values(['model_preds'], ascending = False)
backtest_live = backtest.drop(columns=['f_id', 'id','f_place'])
def bet_or_nobet(f_pm_01m, model_preds):
    if f_pm_01m < 50 and model_preds > 0.9:
        return "BET"
    else:
        return "NO BET"

backtest_live['bet'] = backtest_live.apply(lambda row: bet_or_nobet(row['f_pm_01m'], row['model_preds']), axis=1)

1140/1140 [==============================] - 0s 180us/step


/var/folders/w6/wdkqbmf17b90n_r35p0cb7x00000gn/T/ipykernel_64137/3957409348.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backtest['model_preds'] = y_pred[:, 0:1]
/var/folders/w6/wdkqbmf17b90n_r35p0cb7x00000gn/T/ipykernel_64137/3957409348.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backtest['model_preds'] = round(backtest['model_preds'],2)


In [7]:
backtest_live

,f_ko,f_track,f_horse,f_pm_01m,01m_profit,model_preds,bet
99801,24/07/2023 19:40,BALLINROBE,Beer With The Boys,46.0,-1.000,1.0,BET
94941,24/06/2023 17:05,DOWN ROYAL,Universally,70.0,-1.000,1.0,NO BET
104497,22/08/2023 20:15,ROSCOMMON,Blackstone Cliff,13.0,-1.000,1.0,BET
72424,26/10/2022 16:35,CURRAGH,Shur Lookit,42.0,-1.000,1.0,BET
72425,26/10/2022 16:35,CURRAGH,Flier,38.0,-1.000,1.0,BET
...,...,...,...,...,...,...,...
78064,21/01/2023 14:47,LINGFIELD,Makinmedoit,10.0,-1.000,0.0,NO BET
78065,21/01/2023 14:47,LINGFIELD,Tequilamockingbird,15.5,-1.000,0.0,NO BET
89998,26/05/2023 19:30,CURRAGH,Rahmi,25.0,23.520,0.0,NO BET
89997,26/05/2023 19:30,CURRAGH,Earls,28.0,-1.000,0.0,NO BET


In [27]:
bets = backtest_live[backtest_live['bet'] == 'BET']

In [28]:
bets.sum()

f_ko           24/07/2023 19:4022/08/2023 20:1526/10/2022 16:...
f_track        BALLINROBEROSCOMMONCURRAGHCURRAGHREDCARGALWAYG...
f_horse        Beer With The BoysBlackstone CliffShur LookitF...
f_pm_01m                                                25793.56
01m_profit                                              208.6228
model_preds                                           900.899963
bet            BETBETBETBETBETBETBETBETBETBETBETBETBETBETBETB...
dtype: object

In [29]:
bets.count()

f_ko           938
f_track        938
f_horse        938
f_pm_01m       938
01m_profit     935
model_preds    938
bet            938
dtype: int64

In [30]:
208.6/938

0.22238805970149253